In [ ]:
# 📦 Install required libraries
!pip install tweepy textblob --quiet
from textblob import download_corpora
download_corpora.download_all()

# 📚 Import required libraries
import re
import tweepy
from textblob import TextBlob

# 🧠 Define TwitterClient class
class TwitterClient:
    def __init__(self):
        '''
        Twitter API credentials setup
        '''
        # Replace the below strings with your own Twitter Developer credentials
        consumer_key = 'YOUR_CONSUMER_KEY'
        consumer_secret = 'YOUR_CONSUMER_SECRET'
        access_token = 'YOUR_ACCESS_TOKEN'
        access_token_secret = 'YOUR_ACCESS_SECRET'

        try:
            self.auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
            self.auth.set_access_token(access_token, access_token_secret)
            self.api = tweepy.API(self.auth)
        except Exception as e:
            print("Error: Authentication Failed\n", e)

    def clean_tweet(self, tweet):
        '''
        Remove mentions, links, and special characters
        '''
        return ' '.join(re.sub(r"(@[A-Za-z0-9_]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def get_tweet_sentiment(self, tweet):
        '''
        Determine sentiment using TextBlob polarity
        '''
        analysis = TextBlob(self.clean_tweet(tweet))
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def get_tweets(self, query, count=100):
        '''
        Fetch tweets and classify sentiment
        '''
        tweets = []
        try:
            fetched_tweets = self.api.search_tweets(q=query, count=count, lang='en')
            for tweet in fetched_tweets:
                parsed_tweet = {
                    'text': tweet.text,
                    'sentiment': self.get_tweet_sentiment(tweet.text)
                }
                if tweet.retweet_count > 0:
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
            return tweets
        except tweepy.TweepyException as e:
            print("Tweepy Error:", str(e))
            return []

# 🚀 Run the analysis
def main():
    print("🔍 Twitter Sentiment Analysis")
    api = TwitterClient()
    topic = input("Enter a keyword/topic to analyze (e.g., 'Python'): ")
    tweets = api.get_tweets(query=topic, count=200)

    if not tweets:
        print("No tweets found or error in fetching.")
        return

    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    netweets = [tweet for tweet in tweets if tweet['sentiment'] == 'neutral']

    # 📊 Show stats
    print(f"\n✅ Positive tweets: {100*len(ptweets)/len(tweets):.2f}%")
    print(f"❌ Negative tweets: {100*len(ntweets)/len(tweets):.2f}%")
    print(f"😐 Neutral tweets : {100*len(netweets)/len(tweets):.2f}%")

    # 📝 Show sample tweets
    print("\n📢 Sample Positive Tweets:")
    for tweet in ptweets[:5]:
        print('-', tweet['text'])

    print("\n📢 Sample Negative Tweets:")
    for tweet in ntweets[:5]:
        print('-', tweet['text'])

# ✅ Entry point
if __name__ == "__main__":
    main()
